In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv')
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['v1'])
df['v1_transformed'] =le.transform(df['v1'])

df.drop(columns='v1', inplace=True)

df['len_v2']=df['v2'].apply(len)
import seaborn as sns
sns.displot(df['len_v2'])
sns.displot(x='len_v2', data=df, hue='v1_transformed')
sns.countplot(x='v1_transformed', data=df)

import re
def preprocess_text(x):
    cleaned_text = re.sub(r'[^a-zA-Z\d\s]+', '', x)
    return cleaned_text.lower()
df['v2_cleaned']= df['v2'].apply(preprocess_text)



from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
tf_idf_transformer = TfidfTransformer(smooth_idf=False)
count_vectorizer.fit(df['v2_cleaned'].values)
count_vect_train = count_vectorizer.transform(df['v2_cleaned'].values)
tf_idf_transformer.fit(count_vect_train)
tf_idf_train = tf_idf_transformer.transform(count_vect_train)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tf_idf_train,df['v1_transformed'] )

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), accuracy_score(y_test, y_pred)

In [ ]:
# Just a better way
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('f1-score: ', f1_score(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
model = LogisticRegression()
max_iter = [100, 200, 500, 1000]
C = [0.1, 0.5, 1, 10, 50, 100]
param_grid = dict(max_iter=max_iter, C=C)
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    cv=5,
                    scoring=['f1'],
                    refit='f1',
                    verbose=2)
grid_result = grid.fit(X_train, y_train)

In [ ]:
grid_result.best_estimator_

In [ ]:
grid_result.best_params_

In [ ]:
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('f1-score: ', f1_score(y_test, y_pred))

Notice the improvement for recall and f1-score with optimized hyperparamters.